In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# 데이터 생성
movies = pd.DataFrame({
    'title': ['The Godfather', 'Pulp Fiction', 'The Dark Knight', 'Fight Club', 
              'Inception', 'The Matrix', 'Goodfellas', 'Interstellar'],
    'genres': ['Crime Drama', 'Crime Drama', 'Action Crime', 'Drama Thriller',
               'Sci-Fi Thriller', 'Action Sci-Fi', 'Crime Drama', 'Sci-Fi Drama']
})

# 사용자-영화 평점 매트릭스 (NaN = 아직 안봄)
ratings = pd.DataFrame({
    'The Godfather': [5, 4, np.nan, 3],
    'Pulp Fiction': [4, 5, 2, np.nan], 
    'The Dark Knight': [np.nan, 4, 5, 4],
    'Fight Club': [3, np.nan, 4, 5],
    'Inception': [5, 3, np.nan, 4],
    'The Matrix': [4, np.nan, 3, 5],
    'Goodfellas': [np.nan, 5, 2, 3],
    'Interstellar': [4, 2, np.nan, np.nan]
}, index=['Alice', 'Bob', 'Carol', 'Dave'])


In [4]:
# 1. 콘텐츠 기반 추천 (장르 유사도)
def content_recommend(movie_title, top_n=2):
    vectorizer = CountVectorizer()
    genre_matrix = vectorizer.fit_transform(movies['genres'])
    similarity = cosine_similarity(genre_matrix)
    
    idx = movies[movies['title'] == movie_title].index[0]
    sim_scores = list(enumerate(similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    recommendations = [movies.iloc[i[0]]['title'] for i in sim_scores[1:top_n+1]]
    return recommendations

In [5]:
# 2. 협업 필터링 추천
def collaborative_recommend(user_name, top_n=2):
    user_ratings = ratings.loc[user_name]
    unseen_movies = user_ratings[user_ratings.isna()].index.tolist()
    
    predictions = {}
    for movie in unseen_movies:
        avg_rating = ratings[movie].mean()
        if not np.isnan(avg_rating):
            predictions[movie] = avg_rating
    
    sorted_preds = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    return [movie for movie, score in sorted_preds[:top_n]]

# 실행
print("영화 추천시스템")
print("데이터:", list(movies['title']))
print()

print("콘텐츠 기반: 'The Dark Knight'와 유사한 영화")
print(content_recommend('The Dark Knight'))
print()

print("협업 필터링: Alice에게 추천")
print(collaborative_recommend('Alice'))

영화 추천시스템
데이터: ['The Godfather', 'Pulp Fiction', 'The Dark Knight', 'Fight Club', 'Inception', 'The Matrix', 'Goodfellas', 'Interstellar']

콘텐츠 기반: 'The Dark Knight'와 유사한 영화
['The Godfather', 'Pulp Fiction']

협업 필터링: Alice에게 추천
['The Dark Knight', 'Goodfellas']
